# Cargar y explorar los datos:

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score

# If necessary, install required packages (use pip in the terminal)
# pip install matplotlib scikit-learn pandas
# pip install pandas-profiling

# 2. Cargar el dataset
df_train = pd.read_csv('./dataset/train.csv')  # Change this to the correct local path
df_test = pd.read_csv('./dataset/test.csv')


In [23]:
#log into wandb
import wandb
wandb.init(
    project="OBL-MACHINE-LEARNING-2024",
    name="DecTree#1"
)

In [7]:
# Resumen de la información del dataframe
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11030 entries, 0 to 11029
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 11030 non-null  int64  
 1   Game Title         11030 non-null  object 
 2   Year               11014 non-null  float64
 3   Publisher          9992 non-null   object 
 4   North America      11030 non-null  float64
 5   Europe             11030 non-null  float64
 6   Japan              11030 non-null  float64
 7   Rest of World      11030 non-null  float64
 8   Global             11030 non-null  float64
 9   Number of Reviews  11030 non-null  object 
 10  Summary            1098 non-null   object 
 11  Wishlist           11030 non-null  object 
 12  Platform           11030 non-null  object 
 13  Genre              11030 non-null  object 
 14  Rating             11030 non-null  float64
dtypes: float64(7), int64(1), object(7)
memory usage: 1.3+ MB


,id,Game Title,Year,Publisher,North America,Europe,Japan,Rest of World,Global,Number of Reviews,Summary,Wishlist,Platform,Genre,Rating
0,1,Bloodstained: Curse of the Moon,2007.0,NaN,0.08,0.02,0.00,0.01,0.17,341.0,“Bloodstained: Curse of the Moon” is packed wi...,397.0,2600,Platform,7.345609
1,2,Plants vs. Zombies,2007.0,NaN,0.08,0.02,0.00,0.01,0.17,719.0,"Zombies are invading your home, and the only d...",82.0,2600,Puzzle,6.511195
2,3,Zero Escape: Zero Time Dilemma,2007.0,NaN,0.08,0.02,0.00,0.01,0.17,350.0,Zero Time Dilemma is the third and final entry...,402.0,2600,Puzzle,6.651093
3,4,Zelda II: The Adventure of Link,1987.0,Nintendo,2.19,0.50,1.61,0.08,4.38,756.0,Zelda II: The Adventure of Link is the second ...,290.0,NES,Adventure,7.900000
4,5,Yume Nikki,2007.0,NaN,0.08,0.02,0.00,0.01,0.17,770.0,Yume Nikki is a 32-Bit freeware game created b...,771.0,2600,Adventure,7.152074


In [24]:
missing_values = df_train.isnull().sum()
print(missing_values)


id                      0
Game Title              0
Year                   16
Publisher            1038
North America           0
Europe                  0
Japan                   0
Rest of World           0
Global                  0
Number of Reviews       0
Summary              9932
Wishlist                0
Platform                0
Genre                   0
Rating                  0
dtype: int64


# Rellenar valores faltantes:
En Publisher, rellenamos con una categoría como "Unknown" si no es crucial.
Para Summary, la eliminamos ya que no es relevante para la clasificación o utilizar técnicas de procesamiento de texto si planeas incluirla.

In [25]:
# Rellenar valores faltantes
def clean_data(df):
    # Fill missing 'Year' values with the median of the column
    df['Year'] = df['Year'].fillna(df['Year'].median())
    
    # Fill missing 'Publisher' values with 'Unknown'
    df['Publisher'] = df['Publisher'].fillna('Unknown')
    
    # Drop the 'Summary' column if it exists
    if 'Summary' in df.columns:
        df = df.drop(columns=['Summary'])
    
    # Check and return remaining missing values
    missing_values = df.isnull().sum()
    
    # Print missing values
    print("Missing values after cleaning:")
    print(missing_values)



    
    return df


# Convertir variables categóricas:

In [26]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def preprocess_data(df, is_train=True):    
    # Limpiar los datos
    df = clean_data(df)
    
    # Función auxiliar para convertir cadenas con 'K' a valores numéricos
    def convert_to_numeric(value):
        value_str = str(value)
        if 'K' in value_str:
            return int(float(value_str.replace('K', '')) * 1000)
        else:
            return int(float(value_str))
    
    # Aplicar la conversión a columnas numéricas
    columns_to_convert = [
        'Europe', 'Japan', 'Rest of World', 'North America', 
        'Global', 'Number of Reviews', 'Wishlist'
    ]
    for column in columns_to_convert:
        if column in df.columns:
            df[column] = df[column].apply(convert_to_numeric)
    
    # Codificación de variables categóricas
    label_encoder = LabelEncoder()
    categorical_columns = ['Game Title', 'Publisher', 'Platform', 'Genre']
    for column in categorical_columns:
        if column in df.columns:
            df[column] = label_encoder.fit_transform(df[column])
    
    # Separar características y objetivo en el conjunto de entrenamiento
    if is_train:
        # Si es el conjunto de entrenamiento, devolver características y objetivo
        X = df.drop(columns=['id', 'Rating'], errors='ignore')
        y = df['Rating'] if 'Rating' in df.columns else None
        return X, y
    else:
        # Si es el conjunto de prueba, eliminar 'id' y 'Rating' y devolver solo características
        X = df.drop(columns=['id', 'Rating'], errors='ignore')
        return X

# Ejemplo de uso:
# X_train, y_train = preprocess_data(df_train, is_train=True)
# X_test = preprocess_data(df_test, is_train=False)


# Separar características y objetivo

In [28]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

def find_best_depth_classification(X, Y, test_size=0.33, max_depth=20, cv=5, random_state=42):
    """
    Finds the best max_depth for a DecisionTreeClassifier using cross-validation.
    """
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)
    depth_range = range(1, max_depth + 1)
    mean_scores = []

    for depth in depth_range:
        model = DecisionTreeClassifier(max_depth=depth, random_state=random_state)
        scores = cross_val_score(model, X_train, Y_train, cv=cv, scoring='accuracy')  # Use accuracy for classification
        mean_scores.append(np.mean(scores))

    best_depth = depth_range[np.argmax(mean_scores)]
    best_score = max(mean_scores)

    print(f"Best depth: {best_depth}")
    print(f"Average Accuracy with cross-validation: {best_score:.4f}")

    return best_depth, best_score




# Hacer arbol y evaluarlo:

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split


def train_and_evaluate_decision_tree_classification(X, Y, best_depth, test_size=0.33, random_state=42):
    """
    Train and evaluate the decision tree for classification.
    """
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)
    model = DecisionTreeClassifier(max_depth=best_depth, random_state=random_state)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(Y_test, Y_pred)
    precision = precision_score(Y_test, Y_pred, average='weighted')
    recall = recall_score(Y_test, Y_pred, average='weighted')
    f1 = f1_score(Y_test, Y_pred, average='weighted')
    conf_matrix = confusion_matrix(Y_test, Y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")

    # Log metrics to wandb
    wandb.log({
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "best_depth": best_depth
    })

    # Plot decision tree
    plt.figure(figsize=(15, 10))
    tree.plot_tree(model, filled=True, feature_names=X.columns, class_names=np.unique(Y).astype(str), fontsize=10)
    plt.title(f"Decision Tree (Depth = {best_depth})")
    plt.show()

    return model





Correr todas las funciones

In [30]:
def run_decision_tree_workflow_classification(df_train, df_test):
    """
    Workflow to preprocess, train, and evaluate a Decision Tree Classifier.
    """
    # Preprocess the data
    X_train, Y_train = preprocess_data(df_train, is_train=True)
    X_test = preprocess_data(df_test, is_train=False)

    # Find the best depth
    best_depth, best_score = find_best_depth_classification(X_train, Y_train)

    # Train and evaluate the model with the best depth
    best_model = train_and_evaluate_decision_tree_classification(X_train, Y_train, best_depth)

    return best_model

Evaluar test

In [32]:
def run_model_on_test_classification(best_model, df_test):
    """
    Run predictions on the test dataset with the trained model.
    """
    X_test = preprocess_data(df_test, is_train=False)
    predictions = best_model.predict(X_test)

    return predictions


# Run the workflow
workflow_results = run_decision_tree_workflow_classification(df_train, df_test)

# Get predictions for the test set
best_model = workflow_results
df_results = run_model_on_test_classification(best_model, df_test)

# Save submission
submission = pd.DataFrame({
    'id': df_test['id'],  # Ensure the test dataset contains an 'id' column
    'Prediction': df_results
})

submission.to_csv('submission#2.csv', index=False)

Missing values after cleaning:
id                   0
Game Title           0
Year                 0
Publisher            0
North America        0
Europe               0
Japan                0
Rest of World        0
Global               0
Number of Reviews    0
Wishlist             0
Platform             0
Genre                0
Rating               0
dtype: int64
Missing values after cleaning:
id                   0
Game Title           0
Year                 0
Publisher            0
North America        0
Europe               0
Japan                0
Rest of World        0
Global               0
Number of Reviews    0
Wishlist             0
Platform             0
Genre                0
dtype: int64


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Karina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Karina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Karina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra

In [ ]:
# Crear un DataFrame para las predicciones

#evaluar nuestro modelo con test


In [ ]:
from joblib import dump, load
dump(best_model, 'tree.joblib')


Importamos librerias

In [ ]:
from joblib import dump, load
dump(best_model, 'tree.joblib')

#model.load('tree.joblib')
#model.predict(X_test)
 #  Guardar el modelo en un archivo